# ***Client Side***

# <a id='index'>Index</a>

+ <a href='#connection'>1. Connecting Client and Server</a>
+ <a href='#client_keys'>2. Generating Client Keys</a>
+ <a href='#exchange'>3. Exchange of Public Keys for encrypted communication</a>
    + <a href='#sending_client_keys_to_server'>3.1 Sending Client Public Keys to Server</a>
    + <a href='#server_keys'>3.2 Receiving Server Public Keys from Server</a>
+ <a href='#communicate'>5. Encrypted Communication</a>

In [1]:
import socket
import numpy as np
import gmpy2
from gmpy2 import powmod,mpz,isqrt,invert
import pyecm
from el_gamal import generate_keys,get_blocks,format_plain_text,format_decrypted_plain_text,encrypt,decrypt

# <a id='connection'>1. Connecting Client and Server</a>
<a href='#index'>Go back to top</a>

In [2]:
host = 'localhost'    
port = 12345                   # The same port as used by the server
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((host, port))

# <a id='client_keys'>2. Generating Client Keys</a>
<a href='#index'>Go back to top</a>

In [3]:
client_e1,client_e2,client_p,client_d = generate_keys()
print('----------------------------------------GENERATED BY CLIENT-------------------------------------------')
print('Client public keys : e1 :',client_e1,'e2 :',client_e2,'p :',client_p)
print('Client secret keys : d:',client_d)
print('-----------------------------------------------------------------------------------')

----------------------------------------GENERATED BY CLIENT-------------------------------------------
Client public keys : e1 : 26 e2 : 432299446094075835731962 p : 1000000000000000000885721
Client secret keys : d: 32
-----------------------------------------------------------------------------------


# <a id='exchange'>3. Exchange of Public Keys for encrypted communication</a>
<a href='#index'>Go back to top</a>

## <a id='sending_client_keys_to_server'>3.1 Sending Client Public Keys to Server</a>

In [4]:
client_public_keys = str(client_e1) + ' ' + str(client_e2) + ' ' + str(client_p)
s.sendall((client_public_keys).encode())

## <a id='server_keys'>3.2 Receiving Server Public Keys from Server</a>

In [5]:
server_public_keys = s.recv(1024)
server_public_keys = (server_public_keys.decode('utf-8')).split(" ")
server_public_keys[0] = int(server_public_keys[0])
server_public_keys[1] = int(server_public_keys[1])
server_public_keys[2] = int(server_public_keys[2])
server_public_keys = (server_public_keys[0],server_public_keys[1],server_public_keys[2])
print('Server Public Keys sent by Server: e1 :', server_public_keys[0],'e2 :',server_public_keys[1],'p :',server_public_keys[2])

Server Public Keys sent by Server: e1 : 24 e2 : 406818960309392683137532 p : 1000000000000000000826893


# <a id='communicate'>5. Encrypted Communication</a>
<a href='#index'>Go back to top</a>

In [ ]:
print('\n------------------------------MESSAGES BETWEEN CLIENT AND SERVER ARE NOW ENE-TO-END ENCRYPTED---------------------------')
while True :
    PT = input('Client : ')
    PT = get_blocks(PT)
    plain_text_blocks = format_plain_text(PT)
    cipher_text_blocks = encrypt(plain_text_blocks,server_public_keys)
    for block in cipher_text_blocks:
        temp = str(block[0]) + ' ' + str(block[1])
        s.sendall(temp.encode())
    temp = 'END'
    s.sendall(temp.encode())
    if PT [0] == 'BYE':
        break
    cipher_text_blocks = []
    while True :
        data = ''
        data = s.recv(1024)
        data = data.decode('utf-8')
        if data == 'END':
            break
        data = data.split(" ")
        cipher_text_blocks.append((mpz(int(data[0])),mpz(int(data[1]))))
    decypted_plain_text_blocks = decrypt(cipher_text_blocks,client_d,(client_e1,client_e2,client_p))
    plain_text_after_decryption = format_decrypted_plain_text(decypted_plain_text_blocks)
    print('Server :',plain_text_after_decryption)
    if plain_text_after_decryption == 'BYE':
        break
s.close()


------------------------------MESSAGES BETWEEN CLIENT AND SERVER ARE NOW ENE-TO-END ENCRYPTED---------------------------
Client : hello server
Server : hello client
